In [4]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
from sklearn.decomposition import PCA
sys.path.append('scripts/')
from ml_method import *
from proj1_helpers import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Used to check the accuracy

In [6]:
y, x, ind = load_csv_data('verify.csv')

mask = (x != -999)

w = np.array([ 1.75815226e-01,  -2.71859587e-01,   5.64233606e-01,
         -7.61631793e-03,  -5.90937217e-01,  -2.94559751e-01,
          3.17968545e-01,  -6.59083445e-02,   4.11845940e-03,
          1.35124053e-02,   5.22805311e-01,   4.98332439e-01,
         -8.87872554e-03,   6.53442720e-01,  -8.71039998e-01,
          5.49099703e-01,  -1.30072281e+00,  -1.20590897e+00,
          4.43600235e-01,  -1.04761735e-01,   1.15319875e-06])
# compute the mean and standard deviations
mean = (x * mask).sum(axis=0)/np.sum(mask, axis=0)
std_dev = np.sqrt((((x - mean) * mask)**2).sum(axis=0)/np.sum(mask, axis=0))

# normalize the data
stand_x = (x * mask - mean)/std_dev

# set -999 to 0
stand_x[~mask] = 0

# stand_x = np.delete(stand_x, [11,12,15,18,20,22,25,26,28], 1)

pca = PCA(n_components = 25)
stand_x = pca.fit_transform(stand_x)
# stand_x.shape

# stand_x = np.delete(stand_x, [14, 15, 17, 18, 20, 24, 25, 28], 1)
# stand_x = np.delete(stand_x, [4,5,6,12,23,24,25,26,27,28], 1)

y_pred = predict_labels(w, stand_x)

create_csv_submission(ind, y_pred, 'prediction.csv')
# ws.shape

frame = pd.read_csv('verify.csv')
frame = frame[['Id', 'Prediction']]
frame['Prediction1'] = pd.Series(y_pred)
frame.Prediction = frame.Prediction.apply(lambda x: -1.0 if x == 'b' else 1.0)
frame['Hit'] = [1 if x == y else 0 for x,y in zip(frame.Prediction, frame.Prediction1)]
sum(frame.Hit)/len(frame)

ValueError: shapes (246,25) and (21,) not aligned: 25 (dim 1) != 21 (dim 0)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

clf = svm.SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

## generate data for submission

In [59]:
y, x, ind = load_csv_data('higgs-data/test.csv')

mask = (x != -999)

w = np.array([ 0.0618007 , -0.67451767, -1.17145257,  0.3153572 , -0.07961639,
         0.80775918,  0.08343444,  1.15862536, -0.06369184,  0.16268409,
        -0.80057822,  0.27119628,  0.39992783,  0.6221701 , -0.00533122,
        -0.00564285,  1.14219461,  0.00151573,  0.00519025,  0.18439471,
        -0.00234673, -0.14334519,  0.20265211, -0.14199682,  0.00344027,
        -0.00235626, -0.28977502,  0.00877695, -0.00408017, -0.20828267])
# compute the mean and standard deviations
mean = (x * mask).sum(axis=0)/np.sum(mask, axis=0)
std_dev = np.sqrt((((x - mean) * mask)**2).sum(axis=0)/np.sum(mask, axis=0))

# normalize the data
stand_x = (x * mask - mean)/std_dev

# set -999 to 0
stand_x[~mask] = 0

# stand_x = np.delete(stand_x, [14, 15, 17, 18, 20, 24, 25, 28], 1)
# stand_x = np.delete(stand_x, [4,5,6,12,23,24,25,26,27,28], 1)

y_pred = predict_labels(w, stand_x)

create_csv_submission(ind, y_pred, 'prediction.csv')

## use panda to load the train data

In [41]:

frame = pd.read_csv('higgs-data/train.csv')
# mean_s = frame.DER_mass_MMC[(frame.Prediction == 's') & (frame.DER_mass_MMC != -999)].mean()
# mean_b = frame.DER_mass_MMC[(frame.Prediction == 'b') & (frame.DER_mass_MMC != -999)].mean()
# print(mean_s, mean_b)

frame.Prediction.value_counts()

b    164333
s     85667
Name: Prediction, dtype: int64

## Generate the dataset with equal case

In [54]:
frame1 = frame[frame.Prediction == 's']
frame2 = frame[frame.Prediction == 'b']
# len(frame1)

frame2 = frame2[0:len(frame1)]
frame3 = pd.concat([frame1, frame2])
frame3.to_csv('train1.csv')

In [23]:
frame = pd.read_csv('prediction.csv')
frame1 = pd.read_csv('prediction1.csv')
# frame.Prediction[frame.Prediction == -1].sum()
print(frame.Prediction[frame.Prediction == 1].sum())
print(frame.Prediction[frame.Prediction == -1].sum())
print(frame1.Prediction[frame1.Prediction == 1].sum())
print(frame1.Prediction[frame1.Prediction == -1].sum())

# frame


259637
-308601
283388
-284850


In [24]:
frame = pd.read_csv('prediction.csv')
frame1 = pd.read_csv('prediction1.csv')

frame['Hit'] = [1 if x == y else 0 for x,y in zip(frame.Prediction, frame1.Prediction)]
sum(frame.Hit)/len(frame)

0.33117637328021005

In [27]:
x = np.array([i+1 for i in range(12)]).reshape([3,4])
cov_mat = np.cov([x[0,:],x[1,:],x[2,:]])


eig_val, eig_vec = np.linalg.eig(cov_mat)
eig_val

print(eig_vec)
print(eig_val)
# eig_vec@np.diag(eig_val)@eig_vec.T

eig_vec1 = eig_vec[:,0:2]

x1 = eig_vec1.T@x
cov_mat1 = np.cov(x1)
eig_val1, eig_vec1 = np.linalg.eig(cov_mat1)
print(eig_vec1)
print(eig_val1)
print(eig_vec[0:2]@eig_vec[0:2].T)

[[  5.77350269e-01   7.81476825e-01   2.39086184e-16]
 [  5.77350269e-01  -5.95605390e-01  -7.07106781e-01]
 [  5.77350269e-01  -1.85871435e-01   7.07106781e-01]]
[  5.00000000e+00  -1.50192728e-16   1.23259516e-32]
[[  1.00000000e+00   3.58953035e-16]
 [ -3.58953035e-16   1.00000000e+00]]
[  5.00000000e+00   1.44624499e-31]
[[ 0.94403936 -0.13211848]
 [-0.13211848  1.18807911]]
